<center><h1 style="font-size:2em;color:#2467C0;font-family:Times New Roman">Tagesaufgaben</h1></center>


## Aufgabe 1

1. Erstelle eine Python-Funktion connect_f1(), die sich mit der Formel1-SQLite-Datenbank verbindet die Verbindung zurückgibt. Führe die Funktion anschließend aus. Schließe sie zuletzt.

In [1]:
import sqlite3
import pandas as pd

In [2]:
data_path = r'C:\Users\Admin\OneDrive\Dokumente\DataCraft\DataAnalyst_Aug.24\08_Datenbanken_und_SQL\databases\formula1.sqlite'


def connect_f1():
	try:
		conn = sqlite3.connect(data_path)
		return conn
	except:
		print('something went wrong.')
		return None


conn = connect_f1()


In [6]:
conn.close()

<div>
2. Lass dir die SQL Tabelle drivers in Form einer Pandas Tabelle ausgeben.
</div>

In [7]:
conn = sqlite3.connect(data_path)
df = pd.read_sql_query("SELECT * FROM drivers", conn)
df

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,07/01/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,None,HEI,Nick,Heidfeld,10/05/1977,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,27/06/1985,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,29/07/1981,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,None,KOV,Heikki,Kovalainen,19/10/1981,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
...,...,...,...,...,...,...,...,...,...
837,839,ocon,31,OCO,Esteban,Ocon,17/09/1996,French,http://en.wikipedia.org/wiki/Esteban_Ocon
838,840,stroll,18,STR,Lance,Stroll,29/10/1998,Canadian,http://en.wikipedia.org/wiki/Lance_Stroll
839,841,giovinazzi,36,GIO,Antonio,Giovinazzi,14/12/1993,Italian,http://en.wikipedia.org/wiki/Antonio_Giovinazzi
840,842,gasly,10,GAS,Pierre,Gasly,07/02/1996,French,http://en.wikipedia.org/wiki/Pierre_Gasly


<div>
3. Erstelle eine Python-Funktion get_races(year), die zu dem gegebenen Jahr einen DataFrame mit den
Rennen und Datum zurückgibt
</div>

In [23]:
def get_races(year):
	query = '''
	SELECT name, date FROM races WHERE strftime('%Y', date) = ?
	'''
	conn = sqlite3.connect(data_path)
	df = pd.read_sql_query(query, conn, params=(year,))
	return df

In [27]:
get_races("2009")

,name,date
0,Australian Grand Prix,2009-03-29
1,Malaysian Grand Prix,2009-04-05
2,Chinese Grand Prix,2009-04-19
3,Bahrain Grand Prix,2009-04-26
4,Spanish Grand Prix,2009-05-10
5,Monaco Grand Prix,2009-05-24
6,Turkish Grand Prix,2009-06-07
7,British Grand Prix,2009-06-21
8,German Grand Prix,2009-07-12
9,Hungarian Grand Prix,2009-07-26


<div>
4. Ergänze die Funktion aus der vorherigen Frage, so dass auch der Erstplatzierte mit zurückgegeben wird. Der Name des Erstplatzierten soll als Vor- und Nachname in einer Spalte erscheinen
</div>

In [14]:
def get_races2(year):
	query = '''
	SELECT
		ra.name,
		ra.date,
		d.forename || ',' || d.surname AS first_place
	FROM races AS ra
	JOIN results AS re ON ra.raceId = re.raceId
	JOIN drivers AS d on d.driverId = re.driverId
	WHERE strftime('%Y', ra.date) = ?
	  AND re.position = 1
	'''

	conn = sqlite3.connect(data_path)
	df2 = pd.read_sql_query(query, conn, params=(year,))
	return df2

In [19]:
get_races2("2000")

,name,date,first_place
0,Australian Grand Prix,2000-03-12,"Michael,Schumacher"
1,Brazilian Grand Prix,2000-03-26,"Michael,Schumacher"
2,San Marino Grand Prix,2000-04-09,"Michael,Schumacher"
3,British Grand Prix,2000-04-23,"David,Coulthard"
4,Spanish Grand Prix,2000-05-07,"Mika,Häkkinen"
5,European Grand Prix,2000-05-21,"Michael,Schumacher"
6,Monaco Grand Prix,2000-06-04,"David,Coulthard"
7,Canadian Grand Prix,2000-06-18,"Michael,Schumacher"
8,French Grand Prix,2000-07-02,"David,Coulthard"
9,Austrian Grand Prix,2000-07-16,"Mika,Häkkinen"



<div>
5. Erstelle eine Python-Funktion get_position(driverid, year), die ein DataFrame mit allen Rennen und
den Platzierungen eines Fahrers des gegebenen Jahrs zurückgibt
</div>

In [74]:
def get_position(driverid, year):
	query = f'''
	SELECT
		ra.name,
		ra.date,
		re.position,
		d.forename || ',' || d.surname AS driver_name
	FROM races AS ra
	JOIN results AS re ON ra.raceId = re.raceId
	JOIN drivers AS d on d.driverId = re.driverId
	WHERE re.driverId = ?
	  AND strftime('%Y', ra.date) = ?
	'''

	conn = sqlite3.connect(data_path)
	df3 = pd.read_sql_query(query, conn, params=(driverid, year))
	return df3

In [75]:
get_position(1, "2009")

,name,date,position,driver_name
0,Australian Grand Prix,2009-03-29,,"Lewis,Hamilton"
1,Malaysian Grand Prix,2009-04-05,7,"Lewis,Hamilton"
2,Chinese Grand Prix,2009-04-19,6,"Lewis,Hamilton"
3,Bahrain Grand Prix,2009-04-26,4,"Lewis,Hamilton"
4,Spanish Grand Prix,2009-05-10,9,"Lewis,Hamilton"
5,Monaco Grand Prix,2009-05-24,12,"Lewis,Hamilton"
6,Turkish Grand Prix,2009-06-07,13,"Lewis,Hamilton"
7,British Grand Prix,2009-06-21,16,"Lewis,Hamilton"
8,German Grand Prix,2009-07-12,18,"Lewis,Hamilton"
9,Hungarian Grand Prix,2009-07-26,1,"Lewis,Hamilton"


## Aufgabe 2

<div>
1. Erstelle eine Datenbank mit dem Namen DataCraft
</div>

In [105]:
datacraft_db = sqlite3.connect(
	r"C:\Users\Admin\OneDrive\Dokumente\DataCraft\DataAnalyst_Aug.24\08_Datenbanken_und_SQL\databases\DataCraft.db")

<div>
2. Erstelle in dieser Datenbank eine Tabelle Studenten mit den Spalten Vorname, Nachname und Alter
</div>

In [106]:
cursor = datacraft_db.cursor()

In [107]:
create_table_studenten = """
		CREATE TABLE IF NOT EXISTS studenten(
		"Vorname" VARCHAR,
        "Nachname" VARCHAR,
        "Alter" INT);"""

cursor.execute(create_table_studenten)

<div>
3. Schreibe eine Funktion, mit der du Studenten in deine neue Liste eintragen kannst.
</div>

In [108]:
def add_studenten(vorname, nachname, alter):
	insert_query = """
		INSERT INTO studenten ("Vorname", "Nachname", "Alter")
		VALUES (?, ?, ?);
	"""
	cursor.execute(insert_query, (vorname, nachname, alter))
	datacraft_db.commit()

In [109]:
add_studenten("Mitch", "Maddison", 39)

In [113]:
cursor.close()
conn.close()
datacraft_db.close()

AttributeError: 'str' object has no attribute 'close'